In [60]:
import PySimpleGUI as sg
import mysql.connector
import re
import os

conexao = mysql.connector.connect(
    user='root', password='admin',
    host='127.0.0.1', database='sistemadecadastro',
    auth_plugin='mysql_native_password'
)
cursor = conexao.cursor(prepared=True)

def main():
    janela, login, cadastro, cadastro2, programa = janela_inicial(),None,None,None,None
    while True:



        window,event,values = sg.read_all_windows()

        #Definindo evento para fechamento das janelas 

        """JANELA INICIAL """
        if event == sg.WIN_CLOSED:
            break



        #Definindo comando para voltar para a primeira janela (INICIO)


        if window == login and event == 'Voltar ao inicio':
            login.close()
            janela = janela_inicial()

        if window == cadastro and event == 'Voltar ao inicio':
            cadastro.close()
            janela = janela_inicial()

        if window == cadastro2 and event == 'Voltar ao inicio':
            cadastro2.close()
            janela = janela_inicial()

        #Evento para entrar na tela de login
        if window == janela and event == 'Login':
            janela.close()
            login = tela_login()

        if window == cadastro2 and event == 'Login':
            cadastro2.close()
            login = tela_login()

        #Evento para entrar na tela de  cadastro
        if window == janela and event == 'Cadastrar':
            janela.close()
            cadastro = tela_cadastro()








    #                                 FINAL PySimpleGUI
    #                                 CORPO PRINCIPAL 



        #PARTE CADASTRO

        if event == 'Fazer Cadastro':
            nome = values["nome"]
            email = values["email"]
            senha = values["senha"]
            senha2 = values["senha2"]

            if not re.search('[a-zA-Z0-9_-]+@[a-zA-Z0-9]+\.[a-zA-Z]{1,3}$',email):
                cadastro['mensagem'].update("Digite um email valido")
                continue
            cadastro['mensagem'].update("")
            if senha != senha2:
                cadastro['mensagem'].update("Senhas diferentes, tente novamente")
                continue
            cadastro['mensagem'].update("")

            if email_existe(cursor,email):
                cadastro['mensagem'].update("Email já existente")
                continue
            else:
                inserindo = 'INSERT INTO usuarios(nome,email,senha) VALUES(%s,%s,%s) '
                cadastrando = (nome, email, senha)
                cursor.execute(inserindo, cadastrando)
                conexao.commit()

                cadastro.hide()
                cadastro2 = tela_cadastro2()

        #PARTE LOGIN

        if event == 'Fazer Login':
            email = values["email"]
            senha = values["senha"]

            if email_existe(cursor,email):
                if senha_existe(cursor,senha):
                    login.close()
                    programa = tela_programa()
                else:
                    login['mensagem'].update("Senha incorreta")
            else:
                login['mensagem'].update("Email não cadastrado")
    
    
    window.close()
            
    #FUNÇÕES 
#                      PySimpleGUI - Corpo

#Definindo o conteudo de cada janela 
def janela_inicial():
    leiaute = [
        [sg.Text("SOFTWARE DE CADASTRO/LOGIN", font=(""))],
        [sg.Image('cadastro.png')],
        [sg.Button('Cadastrar'), sg.Button('Login')],
    ]
    return sg.Window('SOFTWARE DE CADASTRO/LOGIN', layout=leiaute,size=(600,600),finalize=True,element_justification='c',font=("Arial", 15))

#Conteudo janelas de login
def tela_login():
    leiaute = [
        [sg.Image('cadastro.png')],
        [sg.Text("Email")],
        [sg.Input(key="email")],
        [sg.Text("Senha")],
        [sg.Input(key="senha",password_char='*')],
        [sg.Text("",key="mensagem")],
        [sg.Button('Fazer Login'), sg.Button('Voltar ao inicio')],
    ]
    return sg.Window('LOGIN', layout=leiaute,size=(600,600),finalize=True,element_justification='c')


#Conteudo janelas de cadastro

def tela_cadastro():
    leiaute = [
        [sg.Image('cadastro.png')],
        [sg.Text("Nome")],
        [sg.Input(key="nome")],
        [sg.Text("Email")],
        [sg.Input(key="email")],
        [sg.Text("Senha")],
        [sg.Input(key="senha",password_char='*')],
        [sg.Text("Repita a senha")],
        [sg.Input(key="senha2",password_char='*')],
        [sg.Button('Fazer Cadastro'), sg.Button('Voltar ao inicio')],
        [sg.Text("",key="mensagem")],
    ]
    return sg.Window('CADASTRO', layout=leiaute,size=(600,600),finalize=True,element_justification='c')

def tela_cadastro2():
    leiaute = [
        [sg.Image('cadastro.png')],
        [sg.Text("Cadastro bem sucedido")],
        [sg.Button('Login'), sg.Button('Voltar ao inicio')],
        [sg.Text("",key="mensagem")],
    ]
    return sg.Window('CADASTRO', layout=leiaute,size=(600,600),finalize=True,element_justification='c')

def tela_programa():
    leiaute = [
        [sg.Image('cadastro.png')],
        [sg.Text("LOGIN BEM SUCEDIDO")],
    ]
    return sg.Window('CADASTRO', layout=leiaute,size=(600,600),finalize=True,element_justification='c')

def email_existe(cursor, email):
    ler_email = 'SELECT email FROM usuarios WHERE email = %s'
    cursor.execute(ler_email, (email,))
    return cursor.fetchone() is not None
    
def senha_existe(cursor, senha):
    ler_senha = 'SELECT senha FROM usuarios WHERE senha = %s'
    cursor.execute(ler_senha, (senha,))
    return cursor.fetchone() is not None
                            #FIM 
if __name__ == "__main__":
    main()
